In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import rotate, resize
import os 
from sklearn.model_selection import train_test_split
 

In [ ]:
train_labels = []   
train_images=[]
folder_path=r"C:\Users\benal\Desktop\ML\ML_Projects\CAT DOG classification\Dog" 

# Loop through each file in the specified folder
for filename in os.listdir(folder_path):
    # Create the complete image path
    image_path = os.path.join(folder_path, filename)   
    # Read the image and normalize its values to [0, 1]
    img = imread(image_path)
    img = img / 255
    # Resize the image to the required shape
    img = resize(img, output_shape=(3, 224, 224), mode='constant', anti_aliasing=True)
    # Append the image to the list of train_images
    train_images.append(img)
    # Append the label 1 (for Dog) to the list of train_labels
    train_labels.append(1)  

# Convert the lists to NumPy arrays
train_x1 = np.array(train_images)
train_y1 = np.array(train_labels)



In [ ]:

folder_path = r"C:\Users\benal\Desktop\ML\ML_Projects\CAT DOG classification\Cat"  
train_images = []   
train_labels = []   

# Loop through each file in the specified folder
for filename in os.listdir(folder_path):
    # Create the complete image path
    image_path = os.path.join(folder_path, filename)   
    # Read the image and normalize its values to [0, 1]
    img = imread(image_path)
    img = img / 255
    # Resize the image to the required shape
    img = resize(img, output_shape=(3, 224, 224), mode='constant', anti_aliasing=True)
    # Append the image to the list of train_images
    train_images.append(img)
    # Append the label 0 (for Cat) to the list of train_labels
    train_labels.append(0)   

# Convert the lists to NumPy arrays
train_x2 = np.array(train_images)
train_y2 = np.array(train_labels) 
 

In [ ]:
train_x2.shape

In [ ]:
train_x = np.concatenate((train_x1, train_x2), axis=0)
train_y = np.concatenate((train_y1, train_y2), axis=0)
random_indices = np.random.permutation(train_x.shape[0])
train_x = train_x[random_indices]
train_y = train_y[random_indices]
train_x.shape

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=13, stratify=train_y)

In [ ]:
train_x.shape

In [ ]:
final_train_x = []
final_train_y = []

# Loop through each image in the original train_x dataset
for i in range(train_x.shape[0]):
    # Append the original image to the final_train_x list
    final_train_x.append(train_x[i])
    
    # Perform image augmentation: 
    # 1. Rotate the image by 45 degrees using the 'rotate' function from an image processing library
    # 2. Append the rotated image to the final_train_x list
    final_train_x.append(rotate(train_x[i], angle=45, mode='wrap'))
    
    # 3. Flip the image left-to-right using 'np.fliplr'
    # 4. Append the flipped image to the final_train_x list
    final_train_x.append(np.fliplr(train_x[i]))
    
    # For each original image, append its corresponding label three times to the final_train_y list
    # This ensures that the augmented images have the same label as the original image
    for j in range(3):
        final_train_y.append(train_y[i])

# Convert the final_train_x and final_train_y lists to NumPy arrays
train_x = np.array(final_train_x)
train_y = np.array(final_train_y)

In [ ]:
import torch

# For train data
 
train_x = torch.from_numpy(train_x)
train_x = train_x.float()
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)

# For test data
 
test_x = torch.from_numpy(test_x)
test_x = test_x.float()
test_y = test_y.astype(int)
test_y = torch.from_numpy(test_y)
train_x.shape

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn as nn
import torchvision.models as models
def CNN_Model(pretrained=True):
    model = models.resnet50(pretrained=pretrained)  # Returns ResNet-50 model with weights trained on ImageNet
    num_ftrs = model.fc.in_features  # Get the number of features output from the last fully connected layer (fc)
    model.fc = nn.Linear(num_ftrs, 2)  # Overwrite the last fully connected layer (fc) with a custom defined layer for binary classification
    return model
model = CNN_Model(pretrained=True)

# specify loss function (categorical cross-entropy loss)
criterion = nn.CrossEntropyLoss() 

# Specify optimizer which performs Gradient Descent
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Learning Scheduler

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset from train_data_tensor and train_label_tensor
train_dataset = TensorDataset(train_x, train_y)

# Create DataLoader for training data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)

# Similarly, create TensorDataset and DataLoader for test data
test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, drop_last=True)

In [ ]:
n_epochs = 10
for epoch in range(1, n_epochs + 1):
    model.train()
    train_loss = 0.0
    # Training
    for batch_x, batch_y in train_loader:
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_x.size(0)
    # Calculate average training loss for the current epoch
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch}/{n_epochs}, Training Loss: {train_loss:.4f}")

In [ ]:
model.eval()
test_loss = 0.0
test_pred = []
from sklearn.metrics import accuracy_score, confusion_matrix
permutation = torch.randperm(test_x.size()[0])
y_pred_list = []
y_test_list=[]
for i in range(0,test_x.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = test_x[indices], test_y[indices]
        if torch.cuda.is_available():
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        output = model(batch_x)
        y_pred_test = torch.argmax(output, dim=1)
        y_pred_list.extend(y_pred_test.squeeze().tolist())
        y_test_list.extend(batch_y.squeeze().tolist())
        
accuracy = accuracy_score(y_test_list, y_pred_list)
print(f"Accuracy: {accuracy}")

In [ ]:
import seaborn as sns
cm = confusion_matrix(y_test_list, y_pred_list)
# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=[str(i) for i in range(len(cm))],
            yticklabels=[str(i) for i in range(len(cm))])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.savefig("confusion_matrix.pdf", format="pdf")
plt.show()
plt.close()